1. Load an example file
2. Show labels using Named Entity Recognition

In [26]:
import pandas as pd

import spacy
from spacy.matcher import PhraseMatcher

from scholarly import scholarly

import requests
import mimetypes
import os

import uuid

import PyPDF2

from spacy import displacy

In [7]:
ROOT_FOLDER = '../../../'

In [8]:
DATA_FOLDER = '../../../data/'

In [9]:
DETECT_ML_MODEL_FILES_FOLDER = '../../../detect_ml_model_files/'

In [15]:
data_df = pd.read_parquet(f'{DATA_FOLDER}detect_ml_models.parquet')

In [16]:
data_df

,title,abstract,year,url,author_id,query,file_name,file_path
0,Linear regression,Linear regression plays a fundamental role in ...,2012,https://www.cs.columbia.edu/~djhsu/coms4771-f2...,"[wcgZZP8AAAAJ, , El-UNYoAAAAJ]",Linear Regression,7cedd2b0-ac03-49fb-9ef1-398b1a5d7fe1.pdf,../../../detect_ml_model_files/7cedd2b0-ac03-4...
1,Introduction to linear regression analysis,Equation (1.2) is called a linear regression m...,2021,http://sutlib2.sut.ac.th/sut_contents/H133678.pdf,"[5PboKNAAAAAJ, , y2eb6cQAAAAJ]",Linear Regression,91374318-7135-4f51-a3bc-720ececd88de.pdf,../../../detect_ml_model_files/91374318-7135-4...
2,Linear regression,"linear regression, a very simple approach for ...",2023,https://datamineaz.org/readings/ISL_chp3.pdf,"[KUIjZqgAAAAJ, bHZf-c8AAAAJ, tQVe-fAAAAAJ, ZpG...",Linear Regression,6d3627b9-6b0b-423c-8898-3185837a7617.pdf,../../../detect_ml_model_files/6d3627b9-6b0b-4...
3,Linear regression,In linear regression the ordinary least square...,2012,,[vBWC0XIAAAAJ],Linear Regression,,None
4,Applied linear regression,Applied linear regression Applied linear regre...,2005,https://www.stat.cmu.edu/~brian/valerie/617-20...,[wlu6jZQAAAAJ],Linear Regression,c7fed73c-8e13-4770-a589-67708aab0b7e.pdf,../../../detect_ml_model_files/c7fed73c-8e13-4...
...,...,...,...,...,...,...,...,...
1229,Generated faces in the wild: Quantitative comp...,"popular systems including Stable Diffusion, Mi...",2022,https://arxiv.org/pdf/2210.00586,[7jTNT1IAAAAJ],Stable Diffusion,0874bbf1-55fa-4df9-8a07-c6c15ac91c84.pdf,../../../detect_ml_model_files/0874bbf1-55fa-4...
1230,What the daam: Interpreting stable diffusion u...,We probe Stable Diffusion to provide insight i...,2022,https://aclanthology.org/2023.acl-long.310.pdf,"[3icAgxMAAAAJ, tQU-BXIAAAAJ, G-2DepMAAAAJ, eJ5...",Stable Diffusion,e1e814fc-7dea-43b0-8049-928fee635bd7.pdf,../../../detect_ml_model_files/e1e814fc-7dea-4...
1231,Red-teaming the stable diffusion safety filter,We conclude that the Stable Diffusion safety f...,2022,https://arxiv.org/pdf/2210.04610,"[d_rilUYAAAAJ, 9e_KfoEAAAAJ, p_aH5fgAAAAJ, Au7...",Stable Diffusion,37f5b622-ff4e-4405-b63e-4d13c5738a88.pdf,../../../detect_ml_model_files/37f5b622-ff4e-4...
1232,Stable video diffusion: Scaling latent video d...,We present Stable Video Diffusion — a latent v...,2023,https://arxiv.org/pdf/2311.15127,"[vud0t5YAAAAJ, EtPn_v4AAAAJ, 1bYESBYAAAAJ]",Stable Diffusion,214899da-c78e-42d2-9a7d-0368daa239d6.pdf,../../../detect_ml_model_files/214899da-c78e-4...


For each query/model load the first file and visualise the matched entities

In [18]:
# drop records where the file_name is Null
data_df = data_df.dropna()

data_df

,title,abstract,year,url,author_id,query,file_name,file_path
0,Linear regression,Linear regression plays a fundamental role in ...,2012,https://www.cs.columbia.edu/~djhsu/coms4771-f2...,"[wcgZZP8AAAAJ, , El-UNYoAAAAJ]",Linear Regression,7cedd2b0-ac03-49fb-9ef1-398b1a5d7fe1.pdf,../../../detect_ml_model_files/7cedd2b0-ac03-4...
1,Introduction to linear regression analysis,Equation (1.2) is called a linear regression m...,2021,http://sutlib2.sut.ac.th/sut_contents/H133678.pdf,"[5PboKNAAAAAJ, , y2eb6cQAAAAJ]",Linear Regression,91374318-7135-4f51-a3bc-720ececd88de.pdf,../../../detect_ml_model_files/91374318-7135-4...
2,Linear regression,"linear regression, a very simple approach for ...",2023,https://datamineaz.org/readings/ISL_chp3.pdf,"[KUIjZqgAAAAJ, bHZf-c8AAAAJ, tQVe-fAAAAAJ, ZpG...",Linear Regression,6d3627b9-6b0b-423c-8898-3185837a7617.pdf,../../../detect_ml_model_files/6d3627b9-6b0b-4...
4,Applied linear regression,Applied linear regression Applied linear regre...,2005,https://www.stat.cmu.edu/~brian/valerie/617-20...,[wlu6jZQAAAAJ],Linear Regression,c7fed73c-8e13-4770-a589-67708aab0b7e.pdf,../../../detect_ml_model_files/c7fed73c-8e13-4...
8,A review on linear regression comprehensive in...,We discuss linear regression and polynomial re...,2020,https://jastt.org/index.php/jasttpath/article/...,"[G9U01kwAAAAJ, aBdgHxkAAAAJ]",Linear Regression,f8d408d7-9c86-4d72-b6af-6538ed46970f.pdf,../../../detect_ml_model_files/f8d408d7-9c86-4...
...,...,...,...,...,...,...,...,...
1229,Generated faces in the wild: Quantitative comp...,"popular systems including Stable Diffusion, Mi...",2022,https://arxiv.org/pdf/2210.00586,[7jTNT1IAAAAJ],Stable Diffusion,0874bbf1-55fa-4df9-8a07-c6c15ac91c84.pdf,../../../detect_ml_model_files/0874bbf1-55fa-4...
1230,What the daam: Interpreting stable diffusion u...,We probe Stable Diffusion to provide insight i...,2022,https://aclanthology.org/2023.acl-long.310.pdf,"[3icAgxMAAAAJ, tQU-BXIAAAAJ, G-2DepMAAAAJ, eJ5...",Stable Diffusion,e1e814fc-7dea-43b0-8049-928fee635bd7.pdf,../../../detect_ml_model_files/e1e814fc-7dea-4...
1231,Red-teaming the stable diffusion safety filter,We conclude that the Stable Diffusion safety f...,2022,https://arxiv.org/pdf/2210.04610,"[d_rilUYAAAAJ, 9e_KfoEAAAAJ, p_aH5fgAAAAJ, Au7...",Stable Diffusion,37f5b622-ff4e-4405-b63e-4d13c5738a88.pdf,../../../detect_ml_model_files/37f5b622-ff4e-4...
1232,Stable video diffusion: Scaling latent video d...,We present Stable Video Diffusion — a latent v...,2023,https://arxiv.org/pdf/2311.15127,"[vud0t5YAAAAJ, EtPn_v4AAAAJ, 1bYESBYAAAAJ]",Stable Diffusion,214899da-c78e-42d2-9a7d-0368daa239d6.pdf,../../../detect_ml_model_files/214899da-c78e-4...


In [21]:
models = data_df['query'].unique().tolist()

In [27]:
nlp = spacy.load("en_core_web_sm", exclude=["ner"])  # Exclude the default NER component
ruler = nlp.add_pipe("entity_ruler", last=True)

# Load the EntityRuler patterns from a file
ruler.from_disk(f"{DATA_FOLDER}ml_entity_ruler_patterns")

In [29]:
for model in models:
    pdf_file_path = data_df.query(f'query == "{model}"')['file_path'].tolist()[0]

    with open(pdf_file_path, "rb") as pdf_file:
            pdf_reader = PyPDF2.PdfReader(pdf_file)
            for page_num, page in enumerate(pdf_reader.pages):
                # Extract text from the page
                text = page.extract_text()
                if text:
                    # Use spaCy to process the text
                    doc = nlp(text)
                    
                    # Extract sentences with ML_METHOD entities
                    filtered_sentences = [
                        sentence.text
                        for sentence in doc.sents
                        if any(ent.label_ == "ML_METHOD" for ent in sentence.ents)
                    ]

                    if len(filtered_sentences) > 0:
                    
                        # Create a new text containing only those sentences
                        filtered_text = " ".join(filtered_sentences)
                        
                        # Create a new Doc object with the filtered text
                        filtered_doc = nlp(filtered_text)
                        
                        # Print the filtered sentences
                        #print("Filtered Sentences:")
                        #for sentence in filtered_doc.sents:
                        #    print(sentence.text)
    
                        options = {"colors": {"ML_METHOD": "linear-gradient(90deg, #aa9cfc, #fc9ce7)"}}
                        displacy.render(filtered_doc, style="ent", options=options, jupyter=True)

/home/reza/miniconda3/envs/ml-lncRNA/lib/python3.12/site-packages/PyPDF2/_cmap.py:142: PdfReadWarning: Advanced encoding /UniGB-UTF16-H not implemented yet
  warnings.warn(


/home/reza/miniconda3/envs/ml-lncRNA/lib/python3.12/site-packages/PyPDF2/_cmap.py:142: PdfReadWarning: Advanced encoding /UniGB-UTF16-H not implemented yet
  warnings.warn(
/home/reza/miniconda3/envs/ml-lncRNA/lib/python3.12/site-packages/PyPDF2/_cmap.py:142: PdfReadWarning: Advanced encoding /UniGB-UTF16-H not implemented yet
  warnings.warn(


/home/reza/miniconda3/envs/ml-lncRNA/lib/python3.12/site-packages/PyPDF2/_cmap.py:142: PdfReadWarning: Advanced encoding /UniGB-UTF16-H not implemented yet
  warnings.warn(


/home/reza/miniconda3/envs/ml-lncRNA/lib/python3.12/site-packages/PyPDF2/_cmap.py:142: PdfReadWarning: Advanced encoding /UniGB-UTF16-H not implemented yet
  warnings.warn(


/home/reza/miniconda3/envs/ml-lncRNA/lib/python3.12/site-packages/PyPDF2/_cmap.py:142: PdfReadWarning: Advanced encoding /UniGB-UTF16-H not implemented yet
  warnings.warn(
/home/reza/miniconda3/envs/ml-lncRNA/lib/python3.12/site-packages/PyPDF2/_cmap.py:142: PdfReadWarning: Advanced encoding /UniGB-UTF16-H not implemented yet
  warnings.warn(
/home/reza/miniconda3/envs/ml-lncRNA/lib/python3.12/site-packages/PyPDF2/_cmap.py:142: PdfReadWarning: Advanced encoding /UniGB-UTF16-H not implemented yet
  warnings.warn(
/home/reza/miniconda3/envs/ml-lncRNA/lib/python3.12/site-packages/PyPDF2/_cmap.py:142: PdfReadWarning: Advanced encoding /UniGB-UTF16-H not implemented yet
  warnings.warn(
/home/reza/miniconda3/envs/ml-lncRNA/lib/python3.12/site-packages/PyPDF2/_cmap.py:142: PdfReadWarning: Advanced encoding /UniGB-UTF16-H not implemented yet
  warnings.warn(


/home/reza/miniconda3/envs/ml-lncRNA/lib/python3.12/site-packages/PyPDF2/_cmap.py:142: PdfReadWarning: Advanced encoding /UniGB-UTF16-H not implemented yet
  warnings.warn(
/home/reza/miniconda3/envs/ml-lncRNA/lib/python3.12/site-packages/PyPDF2/_cmap.py:142: PdfReadWarning: Advanced encoding /UniGB-UTF16-H not implemented yet
  warnings.warn(
/home/reza/miniconda3/envs/ml-lncRNA/lib/python3.12/site-packages/PyPDF2/_cmap.py:142: PdfReadWarning: Advanced encoding /UniGB-UTF16-H not implemented yet
  warnings.warn(
/home/reza/miniconda3/envs/ml-lncRNA/lib/python3.12/site-packages/PyPDF2/_cmap.py:142: PdfReadWarning: Advanced encoding /UniGB-UTF16-H not implemented yet
  warnings.warn(
/home/reza/miniconda3/envs/ml-lncRNA/lib/python3.12/site-packages/PyPDF2/_cmap.py:142: PdfReadWarning: Advanced encoding /UniGB-UTF16-H not implemented yet
  warnings.warn(
/home/reza/miniconda3/envs/ml-lncRNA/lib/python3.12/site-packages/PyPDF2/_cmap.py:142: PdfReadWarning: Advanced encoding /UniGB-UTF16-H

/home/reza/miniconda3/envs/ml-lncRNA/lib/python3.12/site-packages/PyPDF2/_cmap.py:142: PdfReadWarning: Advanced encoding /UniGB-UTF16-H not implemented yet
  warnings.warn(


In [10]:
model_df = pd.read_excel(f'{ROOT_FOLDER}ML-Model-Categorization.ods', sheet_name='Sheet1', usecols=['MODEL'])

In [11]:
model_df

,MODEL
0,Linear Regression
1,Polynomial Regression
2,Ridge Regression
3,Lasso Regression
4,Elastic Net Regression
...,...
97,Claude
98,LLaMA
99,Contrastive Language-Image Pretraining
100,DALL-E


In [12]:
# nlp = spacy.blank("en")
# ruler = nlp.add_pipe("entity_ruler")

#nlp = spacy.load("en_core_web_sm")
#ruler = nlp.add_pipe("entity_ruler", before="ner")

nlp = spacy.load("en_core_web_sm", exclude=["ner"])  # Exclude the default NER component
ruler = nlp.add_pipe("entity_ruler", last=True)

# Load the EntityRuler patterns from a file
ruler.from_disk(f"{DATA_FOLDER}ml_entity_ruler_patterns")

In [13]:
# Test the pipeline
text = "Support Vector Machine, Support Vector Machines, SVM, and S.V.M. are popular machine learning methods."
doc = nlp(text)

# Print detected entities
for ent in doc.ents:
    print(ent.text, ent.label_)


Support Vector Machine ML_METHOD
Support Vector Machines ML_METHOD
SVM ML_METHOD


In [14]:
models = model_df['MODEL'].values.tolist()

Find terms in file to use as labels

In [15]:
def search_in_file(pdf_file_path):
    search_results = dict()
    # Extract text from PDF
    with open(pdf_file_path, "rb") as pdf_file:
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        for page_num, page in enumerate(pdf_reader.pages):
            # Extract text from the page
            text = page.extract_text()
            if text:
                # Use spaCy to process the text
                doc = nlp(text)
                
                # Detect entities
                for ent in doc.ents:
                    if ent.label_ == "ML_METHOD":
                        if ent.text in search_results:
                            search_results[ent.text].append(page_num + 1)  # Store page number (1-indexed)
                        else:
                            search_results[ent.text] = [page_num + 1]  # Store page number (1-indexed)
    
    # Display search results
    for term, pages in search_results.items():
        if pages:
            print(f"'{term}' found on page(s): {set(pages)}")
        #else:
        #    print(f"'{term}' not found in the document.")

In [17]:
file_paths = data_df['file_path'].dropna().values.tolist()

In [18]:
pdf_file_path = file_paths[0]

print(pdf_file_path)

../../detect_ml_model_files/18d9a07e-aaa5-4cbe-a094-ccc035a96392.pdf


In [19]:
with open(pdf_file_path, "rb") as pdf_file:
    pdf_reader = PyPDF2.PdfReader(pdf_file)
    for page_num, page in enumerate(pdf_reader.pages):
        # Extract text from the page
        text = page.extract_text()
        break

In [20]:
doc = nlp(text)

In [21]:
for ent in doc.ents:
    print(ent)

Linear regression
linear regression


In [22]:
#nlp.add_pipe("sentencizer", before="parser")

# Recreate the Doc object
#doc = nlp(text)

for sentence in doc.sents:
    print(sentence.text)

Linear regression
Daniel Hsu (COMS 4771)
Maximum likelihood estimation
One of the simplest linear regression models is the following: (X1,Y1),..., (Xn,Yn),(X,Y)are iid random
pairs taking values in Rd×R, and
Y|X=x∼N(xTw,σ2),x∈Rd.

Here, the vector w∈Rdand scalarσ2>0are the parameters of the model.
(The marginal distribution of
Xis unspeciﬁed.)

Thelog-likelihood of(w,σ2)given (Xi,Yi)
= (xi,yi)fori= 1,...,nis
n/summationdisplay
i=1/braceleftBigg
ln1√
2πσ2−(yi−xT
iw)2
2σ2/bracerightBigg
+T,
whereTis some quantity that does not depend on (w,σ2).
Therefore, maximizing the log-likelihood over
w∈Rd(for anyσ2>0) is the same as minimizing
1
nn/summationdisplay
i=1(xT
iw−yi)2.

So, the maximum likelihood estimator (MLE) ofwin this model is
ˆw∈arg min
w∈Rd1
nn/summationdisplay
i=1(xT
iw−yi)2.

(It is not necessarily uniquely determined.)

Empirical risk minimization
LetPnbe the empirical distribution on(x1,y1),..., (xn,yn)∈Rd×R, i.e., the probability distribution over
Rd×Rwith probability mass f

In [23]:
from spacy import displacy

# Render the entities in a Jupyter Notebook or as HTML
# displacy.render(doc, style="ent", jupyter=True)  # Use jupyter=True if in a notebook

options = {"colors": {"ORG": "linear-gradient(90deg, #aa9cfc, #fc9ce7)", "ML_METHOD": "#f6c5be"}}
displacy.render(doc, style="ent", options=options, jupyter=True)


Lets only select sentences with a ML_METHOD entity


In [24]:
# Extract sentences with ML_METHOD entities
filtered_sentences = [
    sentence.text
    for sentence in doc.sents
    if any(ent.label_ == "ML_METHOD" for ent in sentence.ents)
]

# Create a new text containing only those sentences
filtered_text = " ".join(filtered_sentences)

# Create a new Doc object with the filtered text
filtered_doc = nlp(filtered_text)

# Print the filtered sentences
print("Filtered Sentences:")
for sentence in filtered_doc.sents:
    print(sentence.text)

Filtered Sentences:
Linear regression
Daniel Hsu (COMS 4771)
Maximum likelihood estimation
One of the simplest linear regression models is the following: (X1,Y1),..., (Xn,Yn),(X,Y)are iid random
pairs taking values in Rd×R, and
Y|X=x∼N(xTw,σ2),x∈Rd.



In [25]:
#displacy.render(filtered_doc, style="ent", jupyter=True)  # Use jupyter=True if in a notebook

options = {"colors": {"ML_METHOD": "linear-gradient(90deg, #aa9cfc, #fc9ce7)"}}
displacy.render(filtered_doc, style="ent", options=options, jupyter=True)